#### **setup**

In [1]:
# import libs and env
import json
from openai import OpenAI
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display, Markdown
from ast import literal_eval
#GPT_MODEL = "gpt-4-1106-preview"
EMBEDDING_MODEL = "text-embedding-3-small"
load_dotenv()  

True

In [2]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession

In [3]:
import os
import vertexai
import pandas as pd
from ast import literal_eval
import chromadb
from dotenv import load_dotenv
vertexai.init(project="smart-agent-414407")
from vertexai.preview.generative_models import GenerativeModel
from vertexai.language_models import TextEmbeddingModel
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-pro")
load_dotenv()

True

test

In [4]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

I do not have access to real-time information and cannot provide an up-to-date exchange rate. For the most accurate and current exchange rates, please refer to a reliable financial news source or currency converter.


#### **chroma setup**

#### **function declarations**

In [5]:
# Placeholder function for generating embeddings using OpenAI
def generate_embedding(text) -> list:
    """
    Generates an embedding vector for the given text using OpenAI's embedding API.
    Replace this with an actual call to openai.Embedding.create() in your code.
    """
    """Text embedding with a Large Language Model."""
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")
    embeddings = model.get_embeddings([text])
    for embedding in embeddings:
        vector = embedding.values
    
    return vector

def process_md_files_to_df(folder_path):
    """
    Processes all markdown files in the given folder and compiles them into a DataFrame.
    
    :param folder_path: Path to the folder containing MD files.
    :return: A DataFrame with the columns id, file_path, title, text, title_vector, content_vector, vector_id.
    """
    data = []
    
    for idx, filename in enumerate(os.listdir(folder_path)):
        if filename.endswith(".md"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                title = os.path.splitext(filename)[0]
                title_vector = generate_embedding(title)
                content_vector = generate_embedding(content)
                
                # Append to data list
                data.append({
                    "id": idx + 1,
                    "file_path": file_path,
                    "title": title,
                    "text": content,
                    "title_vector": title_vector,
                    "content_vector": content_vector,
                    "vector_id": idx  # Assuming vector_id starts from 0
                })
    
    # Convert list to DataFrame
    df = pd.DataFrame(data)
    return df

# Example usage
folder_path = "sop-docs/euroclear"

In [6]:
#article_df = process_md_files_to_df(folder_path)
#article_df.to_csv('sop-docs/sop-docs.csv', index=False)

In [7]:
article_df = pd.read_csv('sop-docs/oa-sop-docs.csv')
article_df.head()

,id,file_path,title,text,title_vector,content_vector,vector_id
0,1,sop-docs/euroclear/external-settlement.md,external-settlement,## External settlement\n19/09/2022\n\n### What...,"[-0.0016799656441435218, 0.0029202322475612164...","[-0.001593194087035954, 0.02452816814184189, -...",0
1,2,sop-docs/euroclear/transaction-lifecycle.md,transaction-lifecycle,## What is the lifecycle of transactions?\n\nT...,"[0.004535823594778776, -0.004832559730857611, ...","[-0.018600638955831528, 0.0048954663798213005,...",1
2,3,sop-docs/euroclear/bridge-settlement.md,bridge-settlement,## Bridge settlement\n\n### What is a Bridge s...,"[-0.004899136256426573, 0.007877608761191368, ...","[-0.012175136245787144, 0.008481089025735855, ...",2
3,4,sop-docs/euroclear/status-reporting.md,status-reporting,"## Unmatched, unsettled, alleged reporting\n21...","[-0.024325305595993996, -0.008656425401568413,...","[-0.023916499689221382, 0.01272289827466011, 0...",3
4,5,sop-docs/euroclear/internal-settlement.md,internal-settlement,## What is an internal settlement transaction?...,"[-0.010655367746949196, 0.009483421221375465, ...","[0.008218420669436455, 0.0009066986385732889, ...",4


In [8]:
article_df.title_vector[0]

'[-0.0016799656441435218, 0.0029202322475612164, 0.007287442218512297, -0.0026469531003385782, -0.02751009538769722, 0.026711279526352882, -0.01869509182870388, -0.014203505590558052, -0.011996251530945301, -0.020264694467186928, 0.0223668422549963, 0.01883523538708687, 0.012949224561452866, 0.02001243829727173, -0.018372762948274612, 0.013362647034227848, 0.03206474706530571, -0.013607896864414215, 0.02442694641649723, -0.013292575255036354, -0.02057301066815853, 0.002371922368183732, -0.023081572726368904, -0.007939107716083527, 0.0008614422404207289, -0.014287590980529785, 0.022058527916669846, -0.019900323823094368, 0.013236518017947674, -0.005605724640190601, -0.001396175823174417, -0.0019935357850044966, -0.008016186766326427, -0.027355937287211418, -0.0253098476678133, -0.011926179751753807, -0.0024490009527653456, -0.002653960371389985, 0.0059981257654726505, 0.01572405733168125, 0.004249840043485165, -0.015990329906344414, -0.016368716955184937, 0.006106736604124308, -0.008345

In [9]:
# Read vectors from strings back into a list
article_df['title_vector'] = article_df.title_vector.apply(literal_eval)
article_df['content_vector'] = article_df.content_vector.apply(literal_eval)

# Set vector_id to be a string
article_df['vector_id'] = article_df['vector_id'].apply(str)

In [10]:
type(article_df.title_vector[0])

list

In [11]:
chroma_client = chromadb.EphemeralClient() # Equivalent to chromadb.Client(), ephemeral.
# Uncomment for persistent client
# chroma_client = chromadb.PersistentClient()

In [12]:
import chromadb.utils.embedding_functions as embedding_functions
#vertex_key = os.getenv("VERTEX_API_KEY")
#embedding_function = embedding_functions.GoogleVertexEmbeddingFunction(api_key=vertex_key)

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"), model_name=EMBEDDING_MODEL)

sop_content_collection = chroma_client.create_collection(name='ec-content', embedding_function=embedding_function)
sop_title_collection = chroma_client.create_collection(name='ec-title', embedding_function=embedding_function)

In [13]:
# Add the content vectors
sop_content_collection.add(
    ids=article_df.vector_id.tolist(),
    embeddings=article_df.content_vector.tolist(),
)

# Add the title vectors
sop_title_collection.add(
    ids=article_df.vector_id.tolist(),
    embeddings=article_df.title_vector.tolist(),
)

In [14]:
def search_df(collection, query, max_results, dataframe):
    results = collection.query(query_texts=query, n_results=max_results, include=['distances']) 
    df = pd.DataFrame({
                'id':results['ids'][0], 
                'score':results['distances'][0],
                'title': dataframe[dataframe.vector_id.isin(results['ids'][0])]['title'],
                'content': dataframe[dataframe.vector_id.isin(results['ids'][0])]['text'],
                })
    
    return df

In [15]:
content_query_result = search_df(
    collection=sop_content_collection,
    query="internal settlement",
    max_results=2,
    dataframe=article_df
)
content_query_result.head()

,id,score,title,content
2,2,1.987230,bridge-settlement,## Bridge settlement\n\n### What is a Bridge s...
9,9,1.996543,united-states-market,## United States - Market basics\n\n### Safeke...


In [16]:
# Function to create the desired string from a DataFrame
def search_knowledge(query, articles, n, clearing_house="na", market="na"):
    df = search_df(
    collection=sop_title_collection,
    query=query,
    max_results=n,
    dataframe=articles
    )
    # Initialize an empty string
    result_string = ""
    
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Append the title and content to the result string, followed by a newline character
        result_string += f"{row['title']}\n{row['content']}\n"
    
    return result_string.strip()  # Use strip() to remove the last newline character

#### **function declarations**

In [36]:
search_knowledge_func = FunctionDeclaration(
    name="search_knowledge",
    description="query a vector knowledge base about settlement operations in euroclear in different markets",
    parameters={
    "type": "object",
    "properties": {
        "query": {
            "type": "string",
            "description": "your search query to retrieve knowledge from a vector database"
        },
    },
         "required": [
            "query"
      ]
  },
)

get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location",
    parameters={
        "type": "object",
        "properties": {"location": {"type": "string", "description": "Location"}},
    },
)

get_product_info_func = FunctionDeclaration(
    name="get_product_sku",
    description="Get the SKU for a product",
    parameters={
        "type": "object",
        "properties": {
            "product_name": {"type": "string", "description": "Product name"}
        },
    },
)

get_store_location_func = FunctionDeclaration(
    name="get_store_location",
    description="Get the location of the closest store",
    parameters={
        "type": "object",
        "properties": {"location": {"type": "string", "description": "Location"}},
    },
)

place_order_func = FunctionDeclaration(
    name="place_order",
    description="Place an order",
    parameters={
        "type": "object",
        "properties": {
            "product": {"type": "string", "description": "Product name"},
            "account": {"type": "integer", "description": "Account number"},
            "address": {"type": "string", "description": "Shipping address"},
        },
    },
)

knowledge_tool = Tool(
    function_declarations=[
        get_current_weather_func,
        get_product_info_func,
        get_store_location_func,
        place_order_func,
        search_knowledge_func
    ],
)

weather_tool = Tool(
    function_declarations=[get_current_weather_func],
)

general_tools = Tool(
    function_declarations=[
        get_current_weather_func,
        search_knowledge_func
    ],
)

#### **chat test**

In [37]:
# from vertexai.preview import generative_models


# def generate_text(prompt) -> str:
#     # Initialize Vertex AI
#     vertexai.init(project="smart-agent-414407")

#     generation_config={
#         "max_output_tokens": 2048,
#         "temperature": 0,
#         "top_p": 1,
#         "top_k": 20

#     }

#     safety_settings = [
#         {
#             "category":"HARM_CATEGORY_SEXUALLY_EXPLICIT",
#             "threshold":"BLOCK_NONE"
#         },
#         {
#             "category":"HARM_CATEGORY_DANGEROUS_CONTENT",
#             "threshold":"BLOCK_NONE"
#         },
#         {
#             "category":"HARM_CATEGORY_HATE_SPEECH",
#             "threshold":"BLOCK_NONE"
#         },
#         {
#             "category":"HARM_CATEGORY_HARASSMENT",
#             "threshold":"BLOCK_NONE"
#         },
#     ]

#     # Load the model
#     model = generative_models.GenerativeModel(model_name="gemini-pro",
#     tools=[knowledge_tool],
#     generation_config=generation_config,
#     safety_settings=safety_settings,
#     )

#     responses = model.generate_content(prompt)  

#     text_responses = []
#     for response in responses:
#         print(response.text)
#         text_responses.append(response.text)
#     return "".join(text_responses)

# chat = model.start_chat()


In [38]:
# def generate_text(prompt) -> str:
#     # Initialize Vertex AI
#     vertexai.init(project="smart-agent-414407")

#     # Load the model
#     model = GenerativeModel("gemini-1.0-pro")

#     # Generation config
#     # config = {"max_output_tokens": 1024, "temperature": 0.5, "top_p": 0.5, "top_k": 10}

#     # Safety config
#     # safety_config = {
#     #     generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
#     #     generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
#     #     generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
#     #     generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
#     #     generative_models.HarmCategory.HARM_CATEGORY_UNSPECIFIED: generative_models.HarmBlockThreshold.BLOCK_NONE,
#     # }  

#     # Generate content
#     responses = model.generate_content(
#         prompt,
#         # generation_config=config,
#         # safety_settings=safety_config,
#         tools=[weather_tool]
#     )

#     text_responses = []
#     for response in responses:
#         print(response.text)
#         text_responses.append(response.text)
#     return "".join(text_responses)

# chat = model.start_chat()


In [46]:
model = GenerativeModel("gemini-1.0-pro")

In [47]:
from vertexai.preview.generative_models import (
    HarmCategory, 
    HarmBlockThreshold )
from google.cloud.aiplatform_v1beta1.types.content import SafetySetting

In [55]:
prompt = "how does internal settlement work in euroclear?"

generation_config={
    "max_output_tokens": 2048,
    "temperature": 0,
    "top_p": 1,
    "top_k": 20

}

safety_settings = [
    SafetySetting({
        "category":"HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold":"BLOCK_NONE"
    }),
    SafetySetting({
        "category":"HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold":"BLOCK_NONE"
    }),
    SafetySetting({
        "category":"HARM_CATEGORY_HATE_SPEECH",
        "threshold":"BLOCK_NONE"
    }),
    SafetySetting({
        "category":"HARM_CATEGORY_HARASSMENT",
        "threshold":"BLOCK_NONE"
    }),
]


response = model.generate_content(
    prompt,
    generation_config=generation_config,
    safety_settings=safety_settings,
    tools=[general_tools],
)

response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "search_knowledge"
        args {
          fields {
            key: "query"
            value {
              string_value: "how does internal settlement work in euroclear?"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 53
  candidates_token_count: 13
  total_token_count: 66
}

In [56]:
chat = model.start_chat()
chat.history

[]

In [59]:
prompt = """
what's the weather in paris?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

text: "As an AI chatbot, I do not have real-time capabilities and do not have direct access to the internet to provide you with the current weather conditions. \n\nFor the most up-to-date and accurate weather forecast for Paris, I recommend checking a reputable weather app or website, such as the following:\n\n* The Weather Channel: https://weather.com/\n* AccuWeather: https://www.accuweather.com/\n* Weather Underground: https://www.wunderground.com/\n* Météo-France (French national meteorological service): https://meteofrance.com/\n\nYou can also use search engines like Google or Bing to search for \"weather in Paris\" to get the most relevant and up-to-date results."

In [53]:
response = chat.send_message(prompt)

In [54]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "The current weather in Boston, Massachusetts, United States, as of February 28, 2023, at 10:00 AM EST, is:\n\n* **Temperature:** 38°F (3°C)\n* **Feels like:** 33°F (1°C)\n* **Wind:** 15 mph (24 km/h) from the WNW\n* **Humidity:** 65%\n* **Dew point:** 28°F (-2°C)\n* **Pressure:** 30.13 inHg (1020.6 hPa)\n* **Visibility:** 10 miles (16 km)\n* **Cloud cover:** Mostly cloudy\n* **Precipitation:** None\n* **UV index:** 1 (Low)\n\n**Forecast:**\n\n* **Today:** Mostly cloudy, with a high near 40°F (4°C) and a low around 32°F (0°C).\n* **Tonight:** Partly cloudy, with a low around 28°F (-2°C).\n* **Tomorrow:** Sunny, with a high near 42°F (6°C) and a low around 30°F (-1°C).\n\nPlease note that weather conditions can change rapidly, so it\'s always a good idea to check the latest forecast from a reputable source."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIG